Quelle méthode (BoW ou TF-IDF) donne les meilleurs résultats ? Pourquoi pensez- vous que c’est le cas ?
LR: bow 


Que se passe-t-il pour les metriques si vous diminuez max features ? (Montrez un graphique)

In [1]:
import main



[MLENS] backend: threading
C:\_Code\courses\AI\TP2_IFT3335\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
